In [114]:
from IPython.display import display
import pandas as pd

data_revenus = pd.read_excel('../../data/revenus_communes_2019.xlsx')
data_bars = pd.read_csv('../../data/osm-fr-bars.csv', sep=';')

display(data_revenus, data_bars)

,Unnamed: 0,DGFiP - Bureau des études statistiques en matière fiscale,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IRCOM revenus 2019
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Montants en milliers d'euros (exceptés ceux de...
2,NaN,Dép.,Commune,Libellé de la commune,Revenu fiscal de référence par tranche (en euros),Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Impôt net (total)*,Nombre de foyers fiscaux imposés,Revenu fiscal de référence des foyers fiscaux ...,Traitements et salaires,NaN,Retraites et pensions,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nombre de foyers concernés,Montant,Nombre de foyers concernés,Montant
4,NaN,010,001,L'Abergement-Clémenciat,TOTAL,447,13521.06,738.597,249,9873.482,312,9535.029,161,4017.749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82930,NaN,B31,999,Autres,50 001 à 100 000,443,31302.071,7757.596,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82931,NaN,B31,999,Autres,Plus de 100 000,586,108462.346,31927.764,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82932,NaN,B31,999,Autres,TOTAL,3963,171885.111,45862.369,2589,170115.095,162,3568.91,31,560.458
82933,NaN,"* Impôt sur le revenu émis par voie de rôle, h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Geo Point,Nom,Boissons,Site web,Téléphone,Horaires,Accessible PMR,PMR (description),Toilettes PMR,Id Wikidata,Id Wikipédia,Code postal,other_tags
0,"44.0549704,0.2603921",Au boin coin,NaN,NaN,+33 5 53 65 67 45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""cuisine"": ""tapas"", ""ref:FR:CRTA"": ""RESAQU047..."
1,"47.9094426,1.9076031",Le Diabolo Menthe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""source"": ""survey""}"
2,"48.8020856,2.427514",Café de la Gare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"41.6976671,9.0644945",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"43.788717,4.8299152",Brasserie du Commerce,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13210,"{""addr:housenumber"": ""22"", ""addr:city"": ""Saint..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17928,"44.2761336,5.2758837",Les Glycines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17929,"46.7227908,2.5037666",La Rotonde,NaN,NaN,+33 2 48961610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17930,"43.8339541,4.3534613",La Movida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17931,"46.7776345,4.8488139",Nicéphore Café,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
data_bars = data_bars[["Geo Point", "Code postal"]]


display(data_bars)
print("Rate of undefined values:", data_bars["Code postal"].isna().sum() / len(data_bars["Code postal"]))

,Geo Point,Code postal
0,"44.0549704,0.2603921",NaN
1,"47.9094426,1.9076031",NaN
2,"48.8020856,2.427514",NaN
3,"41.6976671,9.0644945",NaN
4,"43.788717,4.8299152",13210
...,...,...
17928,"44.2761336,5.2758837",NaN
17929,"46.7227908,2.5037666",NaN
17930,"43.8339541,4.3534613",NaN
17931,"46.7776345,4.8488139",NaN


Rate of undefined values: 0.8377850889421736


Since the rate of undefined value is way higher than 0.5, and that GPS coordinates and postal code are a little redundant, we can reasonably delete Code postal column:

In [116]:
data_bars.drop("Code postal", axis=1,  inplace=True)
data_bars

,Geo Point
0,"44.0549704,0.2603921"
1,"47.9094426,1.9076031"
2,"48.8020856,2.427514"
3,"41.6976671,9.0644945"
4,"43.788717,4.8299152"
...,...
17928,"44.2761336,5.2758837"
17929,"46.7227908,2.5037666"
17930,"43.8339541,4.3534613"
17931,"46.7776345,4.8488139"


In [117]:
data_revenus.rename(columns=lambda n : data_revenus[n][2], inplace= True)
data_revenus.drop(index=[0, 1, 2], inplace=True)
data_revenus

,NaN,Dép.,Commune,Libellé de la commune,Revenu fiscal de référence par tranche (en euros),Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Impôt net (total)*,Nombre de foyers fiscaux imposés,Revenu fiscal de référence des foyers fiscaux imposés,Traitements et salaires,NaN,Retraites et pensions,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nombre de foyers concernés,Montant,Nombre de foyers concernés,Montant
4,NaN,010,001,L'Abergement-Clémenciat,TOTAL,447,13521.06,738.597,249,9873.482,312,9535.029,161,4017.749
5,NaN,010,002,L'Abergement-de-Varey,TOTAL,134,4086.725,207.133,78,2949.746,90,2956.904,45,1223.998
6,NaN,010,004,Ambérieu-en-Bugey,0 à 10 000,1868,7591.848,-69.392,100,565.211,843,5412.394,507,4318.454
7,NaN,010,004,Ambérieu-en-Bugey,10 001 à 12 000,504,5529.506,-26.333,n.c.,n.c.,335,4208.639,185,2456.718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82930,NaN,B31,999,Autres,50 001 à 100 000,443,31302.071,7757.596,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82931,NaN,B31,999,Autres,Plus de 100 000,586,108462.346,31927.764,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82932,NaN,B31,999,Autres,TOTAL,3963,171885.111,45862.369,2589,170115.095,162,3568.91,31,560.458
82933,NaN,"* Impôt sur le revenu émis par voie de rôle, h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
data_revenus = data_revenus[data_revenus["Revenu fiscal de référence par tranche (en euros)"] == 'TOTAL ']
data_revenus

,NaN,Dép.,Commune,Libellé de la commune,Revenu fiscal de référence par tranche (en euros),Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Impôt net (total)*,Nombre de foyers fiscaux imposés,Revenu fiscal de référence des foyers fiscaux imposés,Traitements et salaires,NaN,Retraites et pensions,NaN
4,NaN,010,001,L'Abergement-Clémenciat,TOTAL,447,13521.06,738.597,249,9873.482,312,9535.029,161,4017.749
5,NaN,010,002,L'Abergement-de-Varey,TOTAL,134,4086.725,207.133,78,2949.746,90,2956.904,45,1223.998
14,NaN,010,004,Ambérieu-en-Bugey,TOTAL,8746,221764.764,12949.691,4096,159147.282,5854,155356.3,2878,64143.55
15,NaN,010,005,Ambérieux-en-Dombes,TOTAL,958,30187.779,1722.361,552,23091.685,707,23644.275,310,7146.934
16,NaN,010,006,Ambléon,TOTAL,65,1755.827,122.835,37,1367.008,38,889.939,29,715.367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82904,NaN,B31,985,Mayotte,TOTAL,.,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.,n.c.
82905,NaN,B31,986,Wallis-et-Futuna,TOTAL,177,2154.401,116.475,65,1584.289,65,1292.197,28,408.091
82914,NaN,B31,987,Polynésie française,TOTAL,4719,78564.552,3862.458,1632,47183.79,1003,22425.118,1707,41386.6
82923,NaN,B31,988,Nouvelle-Calédonie,TOTAL,3857,38840.074,2932.715,1412,25067.969,932,19572.433,205,2529.241
